<a href="https://colab.research.google.com/github/nanafish/ORS/blob/main/%E4%BC%9A%E5%A0%B4%E3%82%B9%E3%82%B1%E3%82%B8%E3%83%A5%E3%83%BC%E3%83%AB%E4%BD%9C%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================
# 43期スケジュール自動割当（1セル版）
# ・四半期表のLOCKEDはそのまま残す
# ・週ごとの器（行数＋追加30枠）を厳守
# ・同一週に同一都道府県は2本目以降NG
# ・東京は「1ヶ月あたり2本まで」
# ・都道府県ごとの最小週間隔が大きくなるように
#   → 追加割当時に「その県の最小週間隔」が最大になる週を優先
# --------------------------------------------
# 必要ファイル：
#   43期四半期表2025-12-01.xlsx
#   43期地域別会場回数.xlsx
# を Colab にアップロードしてから実行
# ============================================

import pandas as pd, re
from collections import defaultdict

Q_PATH = "43期四半期表2025-12-01.xlsx"
C_PATH = "43期地域別会場回数.xlsx"

# ==== 読み込み ====
q43 = pd.read_excel(Q_PATH, sheet_name="43期　マスタ ")
q42 = pd.read_excel(Q_PATH, sheet_name="42期　マスタ")
counts_raw = pd.read_excel(C_PATH, sheet_name="AJ43期エリア計画")

# ==== 地域別会場回数 → counts_city ====
counts2 = counts_raw.copy()
counts2["AJ回数"] = pd.to_numeric(counts2.iloc[:, 2], errors="coerce")
counts_city = counts2[~counts2.iloc[:,1].isna() & counts2["AJ回数"].notna()].copy()
counts_city.rename(columns={counts_city.columns[0]:"都道府県列",
                            counts_city.columns[1]:"市"}, inplace=True)

def extract_pref_name(s):
    if not isinstance(s,str): return None
    s = s.strip()
    if not s: return None
    m = re.match(r"([^\d\s]+)", s)
    return m.group(1) if m else None

counts_city["都道府県名"] = counts_city["都道府県列"].apply(extract_pref_name)
counts_city["都道府県名"] = counts_city["都道府県名"].ffill()
counts_city["市"] = counts_city["市"].astype(str).str.strip()
counts_city["AJ回数"] = counts_city["AJ回数"].astype(int)

# 都道府県ごとの総K（大きい県から優先して割り当てるため）
pref_total_K = counts_city.groupby("都道府県名")["AJ回数"].sum().to_dict()

# ==== 週リスト（42期マスタから） ====
weeks_col = q42.columns[0]
weeks42_raw = q42[weeks_col].dropna().astype(str).tolist()
weeks = []
for w in weeks42_raw:
    w2 = w.replace("ｗ","w")
    if re.match(r"^\d{1,2}-\d{1,2}w$", w2) and w2 not in weeks:
        weeks.append(w2)
nweeks = len(weeks)
week_index = {w:i for i,w in enumerate(weeks)}

# 週→月
week_month = {i:int(w.split("-")[0]) for w,i in week_index.items()}

# ==== 43期マスタに都道府県名列を付与 ====
def extract_pref_from_code(s):
    if not isinstance(s,str): return None
    s = s.strip()
    if not s: return None
    return re.sub(r"^\d+", "", s).strip()

if "都道府県" in q43.columns:
    q43["都道府県名"] = q43["都道府県"].astype(str).apply(extract_pref_from_code)
else:
    pref_col = [c for c in q43.columns if "都道府" in str(c)]
    if pref_col:
        q43["都道府県名"] = q43[pref_col[0]].astype(str).apply(extract_pref_from_code)
    else:
        q43["都道府県名"] = None

# ==== 都道府県 → 市リスト ====
pref_to_cities = {}
for _,r in counts_city.iterrows():
    pref = r["都道府県名"]
    city = r["市"]
    pref_to_cities.setdefault(pref, set()).add(city)
pref_names = set(pref_to_cities.keys())
shape_vals = ["東","西","北","九州"]

# ==== 四半期表1行 → (都道府県,市) マッピング ====
def map_row_to_city(row):
    wk = row["週"]
    if wk not in week_index:            return None
    if row["形態"] not in shape_vals:   return None

    pref_name = row["都道府県名"]
    area = row["エリア"] if isinstance(row["エリア"], str) else None

    # 都道府県名欠けをエリアで補完
    if (not isinstance(pref_name,str) or pref_name=="nan") and \
       isinstance(area,str) and area in pref_names:
        pref_name = area
    if not isinstance(pref_name,str) or pref_name=="nan":
        return None

    cities = pref_to_cities.get(pref_name, set())
    if not cities:
        return None

    # 1) 完全一致
    if area in cities:
        return (pref_name, area)

    # 2) 部分一致（加古川 vs 明石・加古川 等）
    if isinstance(area,str):
        cand = [c for c in cities if (area in c or c in area)]
        if len(cand) == 1:
            return (pref_name, cand[0])

    # 3) 愛媛の特例
    if pref_name=="愛媛" and area==pref_name:
        text = row["会場"] if isinstance(row["会場"], str) else ""
        if ("しこちゅ" in text or "四国中央" in text) and "今治" in cities:
            return (pref_name, "今治")
        if "松山" in cities:
            return (pref_name, "松山")

    # 4) エリア=都道府県名 かつ 1市だけ
    if area == pref_name and len(cities) == 1:
        return (pref_name, list(cities)[0])

    return None

# ==== locked（既存C列埋まり行） ====
locked_events = []
locked_weeks_by_city = defaultdict(list)
week_locked_count = defaultdict(int)
locked_pref_weeks = defaultdict(set)
pref_month_counts = defaultdict(int)   # (pref, month) -> count（locked初期値）

for idx,row in q43.iterrows():
    if not isinstance(row["週"], str) or row["週"] not in week_index:
        continue
    if row["形態"] not in shape_vals:
        continue
    if not isinstance(row["エリア"], str) or row["エリア"].strip()=="":
        continue

    mapped = map_row_to_city(row)
    if not mapped:
        continue
    pref, city = mapped
    cid  = f"{pref}_{city}"
    widx = week_index[row["週"]]
    month = week_month[widx]

    locked_events.append({"row_index":idx, "week_idx":widx,
                          "week":row["週"], "pref":pref, "city":city})
    locked_weeks_by_city[cid].append(widx)
    week_locked_count[widx] += 1
    locked_pref_weeks[pref].add(widx)
    pref_month_counts[(pref, month)] += 1

for cid in locked_weeks_by_city:
    locked_weeks_by_city[cid] = sorted(set(locked_weeks_by_city[cid]))

# ==== 週ごとのベーススロット（行数） ====
base_slots = {i:0 for i in range(nweeks)}
for _,row in q43.iterrows():
    if not isinstance(row["週"], str) or row["週"] not in week_index:
        continue
    if row["形態"] not in shape_vals:
        continue
    widx = week_index[row["週"]]
    base_slots[widx] += 1

# ==== 追加スロット30（7,8,10,11,1,2各w+1 & 9-2w/12-2w/3-2w+1） ====
extra_slots = {i:0 for i in range(nweeks)}
for w,idx in week_index.items():
    m = int(w.split("-")[0])
    if m in {7,8,10,11,1,2}:
        extra_slots[idx] += 1
for sp in ["9-2w","12-2w","3-2w"]:
    if sp in week_index:
        extra_slots[week_index[sp]] += 1

week_capacity = {i: base_slots[i] + extra_slots[i] for i in range(nweeks)}

# ==== city_meta & locked_weeks_capped（K回までは間隔計算に使う） ====
city_meta = {}
for _,r in counts_city.iterrows():
    pref = r["都道府県名"]
    city = r["市"]
    K    = int(r["AJ回数"])
    cid  = f"{pref}_{city}"
    city_meta[cid] = (pref, city, K)

locked_weeks_capped = {}
for cid,weeks_list in locked_weeks_by_city.items():
    if cid not in city_meta:
        continue
    pref, city, K = city_meta[cid]
    locked_weeks_capped[cid] = sorted(weeks_list)[:K]

# ==== 均等配置用ヘルパー ====
def assign_weeks_even(k, pattern_idx, nweeks):
    if k <= 0:
        return []
    base = [round(nweeks * i / k) for i in range(k)]
    base = [min(max(b,0), nweeks-1) for b in base]
    offset = int(round(nweeks * pattern_idx / (3 * max(1,k))))
    pos = sorted(((b + offset) % nweeks for b in base))
    used=set(); res=[]
    for p in pos:
        while p in used:
            p = (p+1) % nweeks
        used.add(p); res.append(p)
    return sorted(res)

def build_events_for_pattern(pattern_idx):
    events=[]
    for cid,(pref,city,K) in city_meta.items():
        locked_cap = locked_weeks_capped.get(cid, [])
        L_eff = min(len(locked_cap), K)
        remain = max(K - L_eff, 0)
        if remain <= 0:
            continue
        base = assign_weeks_even(K, pattern_idx, nweeks)
        base2 = base.copy()
        for lw in sorted(set(locked_cap))[:L_eff]:
            if not base2: break
            dists = [abs(lw-b) for b in base2]
            j = min(range(len(base2)), key=lambda i: dists[i])
            base2.pop(j)
        for pos in base2[:remain]:
            events.append({"cid":cid, "pref":pref, "city":city,
                           "target_idx":pos, "K_pref":pref_total_K.get(pref,0)})
    return events

# ==== 都道府県ごとの最小週間隔を意識したスケジューラ ====
def min_gap_with_new_week(existing_weeks, new_w, nweeks):
    if not existing_weeks:
        return nweeks
    ws = sorted(set(existing_weeks) | {new_w})
    if len(ws) == 1:
        return nweeks
    diffs = []
    for i in range(len(ws)-1):
        diffs.append(ws[i+1] - ws[i])
    diffs.append(nweeks - ws[-1] + ws[0])  # 末尾→先頭
    return min(diffs)

TOKYO = "東京"
TOKYO_MONTH_LIMIT = 2

def schedule_events_pref_gap(events):
    week_used = week_locked_count.copy()
    city_weeks = {cid:set(locked_weeks_by_city.get(cid, []))
                  for cid in city_meta.keys()}
    pref_weeks = {pref:set(ws) for pref,ws in locked_pref_weeks.items()}
    for pref in pref_to_cities.keys():
        pref_weeks.setdefault(pref, set())

    pref_month = pref_month_counts.copy()  # (pref, month) -> count

    slot_assign = []
    unassigned  = []
    max_shift = nweeks

    # Kの大きい都道府県を優先
    events_sorted = sorted(
        events,
        key=lambda e: (-e["K_pref"], e["target_idx"])
    )

    for e in events_sorted:
        cid  = e["cid"]
        t    = e["target_idx"]
        pref = e["pref"]
        city = e["city"]

        assigned = False
        best_w   = None
        best_gap = -1
        best_shift = None

        for shift in range(0, max_shift+1):
            if shift == 0:
                candidates = [t]
            else:
                candidates = [(t+shift) % nweeks, (t-shift) % nweeks]

            for widx in candidates:
                # 週キャパ
                if week_used.get(widx,0) >= week_capacity[widx]:
                    continue
                # 同一市 同一週NG
                if widx in city_weeks[cid]:
                    continue
                # 同一都道府県 同一週NG
                if widx in pref_weeks[pref]:
                    continue
                # 東京 月2回制限
                m = week_month[widx]
                if pref == TOKYO and pref_month[(pref, m)] >= TOKYO_MONTH_LIMIT:
                    continue

                # この週に入れたときの「その県の最小週間隔」を評価
                gap = min_gap_with_new_week(pref_weeks[pref], widx, nweeks)

                if gap > best_gap or (gap == best_gap and (best_shift is None or shift < best_shift)):
                    best_gap = gap
                    best_w = widx
                    best_shift = shift

        if best_w is not None:
            # この週に確定
            week_used[best_w] = week_used.get(best_w,0) + 1
            city_weeks[cid].add(best_w)
            pref_weeks[pref].add(best_w)
            m = week_month[best_w]
            pref_month[(pref, m)] += 1
            slot_assign.append({"cid":cid,"pref":pref,
                                "city":city,"week_idx":best_w})
            assigned = True
        else:
            unassigned.append(e)

    return slot_assign, week_used, city_weeks, pref_weeks, unassigned

# ==== 最終スケジュール作成 ====
def build_schedule(label, assign):
    rows=[]
    for ev in locked_events:
        rows.append({"案":label,"週":ev["week"],
                     "都道府県名":ev["pref"],
                     "市":ev["city"],
                     "source":"locked"})
    for ev in assign:
        rows.append({"案":label,"週":weeks[ev["week_idx"]],
                     "都道府県名":ev["pref"],
                     "市":ev["city"],
                     "source":"assigned"})
    df = pd.DataFrame(rows)
    df = df.sort_values(["都道府県名","市","週"])
    df["回数番号"] = df.groupby(["都道府県名","市"]).cumcount()+1
    df = df.merge(counts_city[["都道府県名","市","AJ回数"]],
                  on=["都道府県名","市"], how="left")
    df = df.rename(columns={"AJ回数":"AJ回数計画"})
    final_counts = df.groupby(["都道府県名","市"]).size().reset_index(name="最終回数")
    df = df.merge(final_counts, on=["都道府県名","市"], how="left")
    df["week_idx"] = df["週"].map(week_index)
    df = df.sort_values(["week_idx","都道府県名","市","回数番号"])
    return df[["案","週","都道府県名","市","回数番号","AJ回数計画","最終回数","source"]]

# ==== 案A/B/Cを作成 ====
eventsA = build_events_for_pattern(0)
assignA, week_usedA, city_weeksA, pref_weeksA, unassignedA = schedule_events_pref_gap(eventsA)
dfA = build_schedule("案A", assignA)

eventsB = build_events_for_pattern(1)
assignB, week_usedB, city_weeksB, pref_weeksB, unassignedB = schedule_events_pref_gap(eventsB)
dfB = build_schedule("案B", assignB)

eventsC = build_events_for_pattern(2)
assignC, week_usedC, city_weeksC, pref_weeksC, unassignedC = schedule_events_pref_gap(eventsC)
dfC = build_schedule("案C", assignC)

# ==== CSV出力 ====
dfA.to_csv("43期AJスケジュール案A_週スロット厳守_都道府県間隔最大化_Tokyo月2.csv", index=False, encoding="utf-8-sig")
dfB.to_csv("43期AJスケジュール案B_週スロット厳守_都道府県間隔最大化_Tokyo月2.csv", index=False, encoding="utf-8-sig")
dfC.to_csv("43期AJスケジュール案C_週スロット厳守_都道府県間隔最大化_Tokyo月2.csv", index=False, encoding="utf-8-sig")

print("A/B/C 出力完了")


A/B/C 出力完了


In [ ]:
# ============================================
# 43期スケジュール最適化（ILP / OR-Tools・1セル）
# ・四半期表のLOCKEDはそのまま固定
# ・週ごとの器（行数＋追加30枠）を厳守
# ・同一週に同一都道府県の追加開催は1本まで
#   （LOCKEDで複数入っている分はそのまま）
# ・東京は「1ヶ月あたり2本まで」
# ・目的：全都道府県について
#          「イベントどうしの最小週間隔」を最大化
# --------------------------------------------
# 使い方（Colab想定）：
# 1) このセルを貼り付けて実行
# 2) 43期四半期表2025-12-01.xlsx
#    43期地域別会場回数.xlsx
#    をこのノートブックと同じ場所に置く
# 3) 実行後、43期AJスケジュール案_ILP_opt.csv が出力される
# ============================================

!pip install -q ortools openpyxl

import pandas as pd, re
from collections import defaultdict
from ortools.sat.python import cp_model

# ===== ファイル名 =====
Q_PATH = "43期四半期表2025-12-01.xlsx"
C_PATH = "43期地域別会場回数.xlsx"

# ===== データ読み込み =====
q43 = pd.read_excel(Q_PATH, sheet_name="43期　マスタ ")
q42 = pd.read_excel(Q_PATH, sheet_name="42期　マスタ")
counts_raw = pd.read_excel(C_PATH, sheet_name="AJ43期エリア計画")

# ----- 地域別会場回数 -----
counts2 = counts_raw.copy()
counts2["AJ回数"] = pd.to_numeric(counts2.iloc[:,2], errors="coerce")

counts_city = counts2[~counts2.iloc[:,1].isna() & counts2["AJ回数"].notna()].copy()
counts_city.rename(columns={counts_city.columns[0]:"都道府県列",
                            counts_city.columns[1]:"市"}, inplace=True)

def extract_pref_name(s):
    if not isinstance(s,str): return None
    s = s.strip()
    if not s: return None
    m = re.match(r"([^\d\s]+)", s)
    return m.group(1) if m else None

counts_city["都道府県名"] = counts_city["都道府県列"].apply(extract_pref_name)
counts_city["都道府県名"] = counts_city["都道府県名"].ffill()
counts_city["市"] = counts_city["市"].astype(str).str.strip()
counts_city["AJ回数"] = counts_city["AJ回数"].astype(int)

# 都道府県ごとの総K（大きい県から優先するときに使える）
pref_total_K = counts_city.groupby("都道府県名")["AJ回数"].sum().to_dict()

# ----- 週リスト（42期マスタから） -----
weeks_col = q42.columns[0]
weeks42_raw = q42[weeks_col].dropna().astype(str).tolist()
weeks = []
for w in weeks42_raw:
    w2 = w.replace("ｗ","w")
    if re.match(r"^\d{1,2}-\d{1,2}w$", w2) and w2 not in weeks:
        weeks.append(w2)
nweeks = len(weeks)
week_index = {w:i for i,w in enumerate(weeks)}
week_month = {i:int(w.split("-")[0]) for w,i in week_index.items()}

# ----- 43期マスタ：都道府県名列 -----
def extract_pref_from_code(s):
    if not isinstance(s,str): return None
    s = s.strip()
    if not s: return None
    return re.sub(r"^\d+", "", s).strip()

if "都道府県" in q43.columns:
    q43["都道府県名"] = q43["都道府県"].astype(str).apply(extract_pref_from_code)
else:
    pref_col = [c for c in q43.columns if "都道府" in str(c)]
    if pref_col:
        q43["都道府県名"] = q43[pref_col[0]].astype(str).apply(extract_pref_from_code)
    else:
        q43["都道府県名"] = None

# ----- 都道府県→市リスト -----
pref_to_cities = {}
for _,r in counts_city.iterrows():
    pref = r["都道府県名"]
    city = r["市"]
    pref_to_cities.setdefault(pref, set()).add(city)
pref_names = set(pref_to_cities.keys())
shape_vals = ["東","西","北","九州"]

# ===== 四半期表の1行→(都道府県, 市)マッピング =====
def map_row_to_city(row):
    wk = row["週"]
    if wk not in week_index:            return None
    if row["形態"] not in shape_vals:   return None

    pref_name = row["都道府県名"]
    area = row["エリア"] if isinstance(row["エリア"], str) else None

    if (not isinstance(pref_name,str) or pref_name=="nan") and \
       isinstance(area,str) and area in pref_names:
        pref_name = area
    if not isinstance(pref_name,str) or pref_name=="nan":
        return None

    cities = pref_to_cities.get(pref_name, set())
    if not cities:
        return None

    if area in cities:
        return (pref_name, area)

    if isinstance(area,str):
        cand = [c for c in cities if (area in c or c in area)]
        if len(cand) == 1:
            return (pref_name, cand[0])

    if pref_name=="愛媛" and area==pref_name:
        text = row["会場"] if isinstance(row["会場"], str) else ""
        if ("しこちゅ" in text or "四国中央" in text) and "今治" in cities:
            return (pref_name, "今治")
        if "松山" in cities:
            return (pref_name, "松山")

    if area == pref_name and len(cities) == 1:
        return (pref_name, list(cities)[0])

    return None

# ===== LOCKED 抽出 =====
locked_events = []                      # （あとでそのまま出力用）
locked_city_weeks = defaultdict(list)   # city_id -> [week_idx]
week_locked_count = defaultdict(int)    # week_idx -> locked本数
pref_locked_flag = defaultdict(lambda: defaultdict(int)) # pref->week_idx->locked本数
pref_month_counts = defaultdict(int)    # (pref,month) -> count（Tokyo月2制約用）

for idx,row in q43.iterrows():
    if not isinstance(row["週"], str) or row["週"] not in week_index:
        continue
    if row["形態"] not in shape_vals:
        continue
    if not isinstance(row["エリア"], str) or row["エリア"].strip()=="":
        continue

    mapped = map_row_to_city(row)
    if not mapped:
        continue
    pref, city = mapped
    cid  = f"{pref}_{city}"
    widx = week_index[row["週"]]
    month = week_month[widx]

    locked_events.append({"row_index":idx, "week_idx":widx,
                          "week":row["週"], "pref":pref, "city":city})
    locked_city_weeks[cid].append(widx)
    week_locked_count[widx] += 1
    pref_locked_flag[pref][widx] += 1
    pref_month_counts[(pref, month)] += 1

# ===== 週ごとの器（ベース＋追加30枠） =====
base_slots = {i:0 for i in range(nweeks)}
for _,row in q43.iterrows():
    if not isinstance(row["週"], str) or row["週"] not in week_index:
        continue
    if row["形態"] not in shape_vals:
        continue
    widx = week_index[row["週"]]
    base_slots[widx] += 1

extra_slots = {i:0 for i in range(nweeks)}
for w,idx in week_index.items():
    m = int(w.split("-")[0])
    if m in {7,8,10,11,1,2}:
        extra_slots[idx] += 1
for sp in ["9-2w","12-2w","3-2w"]:
    if sp in week_index:
        extra_slots[week_index[sp]] += 1

week_capacity = {i: base_slots[i] + extra_slots[i] for i in range(nweeks)}

# ===== city_meta / city list / pref list =====
city_meta = {}   # cid -> (pref, city, K)
for _,r in counts_city.iterrows():
    pref = r["都道府県名"]
    city = r["市"]
    K    = int(r["AJ回数"])
    cid  = f"{pref}_{city}"
    city_meta[cid] = (pref, city, K)

cities = sorted(city_meta.keys())
prefs  = sorted(pref_to_cities.keys())

city_idx = {cid:i for i,cid in enumerate(cities)}
pref_idx = {p:i for i,p in enumerate(prefs)}

# 各cityのlocked数と、K_eff（lockedがKを超えていたらlocked優先）
locked_city_count = {cid:len(locked_city_weeks.get(cid, [])) for cid in cities}
K_eff = {}
for cid in cities:
    K = city_meta[cid][2]
    K_eff[cid] = max(K, locked_city_count.get(cid,0))

# Pref×week の上限（LOCKED分は許容、それ以上は最大1本）
pref_week_limit = {}
for p in prefs:
    for widx in range(nweeks):
        base = pref_locked_flag[p].get(widx,0)
        if base > 0:
            # すでに複数LOCKEDがあっても、それ以上増やさない
            pref_week_limit[(p,widx)] = base
        else:
            pref_week_limit[(p,widx)] = 1

# ===== ILP（CP-SAT）モデル構築 =====
model = cp_model.CpModel()

# x[c,w] : city c が week w に開催するか
x = {}
for cid in cities:
    for widx in range(nweeks):
        x[(cid,widx)] = model.NewBoolVar(f"x_{cid}_{widx}")

# y[p,w] : prefecture p が week w に少なくとも1本あるか
y = {}
for p in prefs:
    for widx in range(nweeks):
        y[(p,widx)] = model.NewBoolVar(f"y_{p}_{widx}")

# t : 全都道府県に共通の「最小週間隔」の下限
t = model.NewIntVar(0, nweeks, "t")

# ----- LOCKED固定 -----
for cid, weeks_list in locked_city_weeks.items():
    for widx in weeks_list:
        model.Add(x[(cid,widx)] == 1)

# ----- cityごとの総回数 == K_eff -----
for cid in cities:
    model.Add(
        sum(x[(cid,widx)] for widx in range(nweeks)) == K_eff[cid]
    )

# ----- 週キャパ制約 -----
for widx in range(nweeks):
    model.Add(
        sum(x[(cid,widx)] for cid in cities) <= week_capacity[widx]
    )

# ----- y[p,w] と x[c,w] の関係 & 都道府県×週の本数上限 -----
for p in prefs:
    cities_in_p = [cid for cid in cities if city_meta[cid][0] == p]
    for widx in range(nweeks):
        # y[p,w] は、その県のどれか1市でも開催していれば1
        for cid in cities_in_p:
            model.Add(x[(cid,widx)] <= y[(p,widx)])
        # 県としての本数上限（LOCKED分＋追加最大1本）
        model.Add(
            sum(x[(cid,widx)] for cid in cities_in_p) <= pref_week_limit[(p,widx)]
        )

# ----- 東京 月2回制約 -----
TOKYO = "東京"
TOKYO_MONTH_LIMIT = 2
tokyo_cities = [cid for cid in cities if city_meta[cid][0] == TOKYO]

for m in range(1,13):
    weeks_in_month = [widx for widx in range(nweeks) if week_month[widx] == m]
    if not weeks_in_month:
        continue
    model.Add(
        sum(x[(cid,widx)] for cid in tokyo_cities for widx in weeks_in_month)
        <= TOKYO_MONTH_LIMIT
    )

# ----- 「最小週間隔 ≥ t」を表す制約 -----
M = nweeks  # ビッグM
for p in prefs:
    for w1 in range(nweeks):
        for w2 in range(w1+1, nweeks):
            # 両方の週でy=1なら、(w2 - w1) >= t
            # そうでなければ緩和
            model.Add(
                (w2 - w1) + M * (2 - y[(p,w1)] - y[(p,w2)]) >= t
            )

# ----- 目的関数：t を最大化 -----
model.Maximize(t)

# ===== ソルバー実行 =====
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 300  # 必要に応じて伸ばす
solver.parameters.num_search_workers = 8

status = solver.Solve(model)
print("Status:", solver.StatusName(status))
print("最小週間隔 t =", solver.Value(t))

if status not in (cp_model.OPTIMAL, cp_model.FEASIBLE):
    raise RuntimeError("実行条件が厳しすぎて解が見つかりませんでした。制約を少し緩めてください。")

# ===== 解からスケジュールを復元 =====
rows = []

# LOCKED分
for ev in locked_events:
    rows.append({
        "案": "ILP案",
        "週": ev["week"],
        "都道府県名": ev["pref"],
        "市": ev["city"],
        "source": "locked"
    })

# 追加分（locked以外で x=1 になっているもの）
locked_city_week_set = {(cid,w) for cid,ws in locked_city_weeks.items() for w in ws}

for cid in cities:
    pref, city, _ = city_meta[cid]
    for widx in range(nweeks):
        if (cid,widx) in locked_city_week_set:
            continue
        if solver.Value(x[(cid,widx)]) == 1:
            rows.append({
                "案": "ILP案",
                "週": weeks[widx],
                "都道府県名": pref,
                "市": city,
                "source": "assigned"
            })

df = pd.DataFrame(rows)
df = df.sort_values(["都道府県名","市","週"])
df["回数番号"] = df.groupby(["都道府県名","市"]).cumcount()+1
df = df.merge(counts_city[["都道府県名","市","AJ回数"]],
              on=["都道府県名","市"], how="left")
df = df.rename(columns={"AJ回数":"AJ回数計画"})
final_counts = df.groupby(["都道府県名","市"]).size().reset_index(name="最終回数")
df = df.merge(final_counts, on=["都道府県名","市"], how="left")
df["week_idx"] = df["週"].map(week_index)
df = df.sort_values(["week_idx","都道府県名","市","回数番号"])
df = df[["案","週","都道府県名","市","回数番号","AJ回数計画","最終回数","source"]]

out_path = "43期AJスケジュール案_ILP_opt.csv"
df.to_csv(out_path, index=False, encoding="utf-8-sig")
out_path
